In [7]:
import os

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import AdamW, get_scheduler
import torch
from torch.optim import Adam
from datasets import load_metric
import numpy as np


In [8]:
os.listdir()

['sample.csv',
 'merger.py',
 'readme.md',
 'logs',
 'wandb',
 'BERT-classification-train.ipynb',
 '.ipynb_checkpoints',
 'supplement',
 'deberta-sudact-aug',
 'results',
 'merged_output.csv',
 'deberta-sudact']

In [20]:
df = pd.read_csv('sample.csv')

In [28]:
print(df.iloc[100,:].text)
print(df.iloc[100,:]['class'])

По месту требования
От гражданки России Алябьева Анастасия Александровна , 01.01.2000 г.р., пол женский, место рождения: место, загранпаспорт 00№0000000, выдан НЕ ГУ МВД 12.12.2012, (если имеется внутренний паспорт)  зарегистрированной в России по адресу: адрес и временно находящейся в Канаде, (для имеющих штамп в паспорте о ПМЖ) постоянно проживающей в Канаде по адресу: адрес,


З А Я В Л Е Н И Е
(согласие на приватизацию)

Я,Алябьева Анастасия Александровна , 01.01.2000 г.р.., настоящим заявлением даю согласие на приватизацию жилой площади, находящейся по адресу: адрес.
Прошу включить меня в договор приватизации.

Подпись ___________________________________________________________________________

application


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the dataset

# Split the dataset into training and temporary data (test + validation)
train_df, temp_df = train_test_split(df, test_size=0.2, stratify=df['class'], random_state=42)

# Split the temporary dataset into test and validation datasets
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['class'], random_state=42)


In [15]:
# Encode class labels
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

# Split dataset into train, validation, and test sets
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42)
train_df, val_df = train_test_split(train_val_df, test_size=0.1, random_state=42)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepvk/deberta-v1-base")


array(['act'], dtype=object)

In [8]:
class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, labels, max_length=512):
        # Ensure texts are strings
        texts = [str(text) for text in texts]
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length)
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [9]:
# Prepare datasets
train_dataset = TextDataset(tokenizer, train_df['text'].tolist(), train_df['class'].tolist())
val_dataset = TextDataset(tokenizer, val_df['text'].tolist(), val_df['class'].tolist())
test_dataset = TextDataset(tokenizer, test_df['text'].tolist(), test_df['class'].tolist())

# Prepare dataloaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)


In [17]:
model = AutoModelForSequenceClassification.from_pretrained("intfloat/multilingual-e5-base", num_labels=len(label_encoder.classes_))

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # number of training epochs
    per_device_train_batch_size=1,  # batch size for training
    per_device_eval_batch_size=1,   # batch size for evaluation
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy="steps",
    learning_rate = 3e-5,
    eval_steps=100,                  # Evaluation and save happens every 500 steps
    save_total_limit=2,              # Only last 2 models are saved. Older ones are deleted.
    load_best_model_at_end=True,     # Load the best model at the end of training
    fp16=True                        # Use mixed precision
)

In [23]:
from datasets import load_metric

accuracy_metric = load_metric("accuracy")
f1_metric = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average='weighted')
    return {"accuracy": acc['accuracy'], "f1": f1['f1']}


Using the latest cached version of the module from /home/kglinsk/.cache/huggingface/modules/datasets_modules/metrics/f1/1ae7ede7c974ce472c931da73bdb2e3f0e1044a996d2348b328773d44dd1847b (last modified on Fri Apr 12 22:30:42 2024) since it couldn't be found locally at f1, or remotely on the Hugging Face Hub.


In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics)

In [25]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 734.00 MiB. GPU 0 has a total capacty of 5.80 GiB of which 549.56 MiB is free. Including non-PyTorch memory, this process has 5.25 GiB memory in use. Of the allocated memory 4.54 GiB is allocated by PyTorch, and 571.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
model = AutoModelForSequenceClassification.from_pretrained("deberta-sudact")
tokenizer = AutoTokenizer.from_pretrained("deepvk/deberta-v1-base")

In [33]:
from joblib import dump, load
dump(label_encoder, 'encoder.joblib') # save the model

['encoder.joblib']

In [32]:
inputs = tokenizer(str(df.iloc[100,:].text), return_tensors="pt", padding=True, truncation=True)
    
    # Get predictions
with torch.no_grad():
        logits = model(**inputs).logits
    
    # Convert logits to probabilities
probabilities = torch.nn.functional.softmax(logits, dim=-1).flatten().tolist()

    # Get predicted class index
label_encoder.inverse_transform([np.argmax(probabilities)])

array(['application'], dtype=object)